In [8]:
from langchain_core.runnables import RunnableConfig
from langgraph.graph import StateGraph
from pydantic import BaseModel


# 基本的なStateの定義
class State(BaseModel):
    value: str


def node_A(state: State, config: RunnableConfig) -> State:
    return State(value="A")


# StateGraphの定義
graph_builder = StateGraph(State)
graph_builder.add_node(node_A)

graph_builder.set_entry_point("node_A")

graph = graph_builder.compile()

print(graph.invoke(State(value="")))


{'value': 'A'}


# Stateの設計戦略

- refs:
  - [How to define input/output schema for your graph¶](https://langchain-ai.github.io/langgraph/how-tos/input_output_schema/)
  - [LangGraphにおけるStateの分割方法](https://zenn.dev/pharmax/articles/a9b9762c2c384e)

- LangGraphにおけるStateとは?
  - = ワークフロー内の各ノード間で受け渡されるデータ(状態)。
  - Stateの設計次第で、アプリケーションの可読性、拡張性、保守性が大きく変わる。
- Stateの課題
  - 適切に設計しないと、以下のような問題が発生
    - すべてのデータを単一のStateに保持すると、Stateが肥大化。
    - 不要なデータがノードに渡ると、パフォーマンスが低下。
    - どのノードでStateのどのデータが使われているか不明瞭だと、デバッグ困難になり運用コストが増加。
    - 

## Stateを効率的に管理する3種の分割戦略1: InputとOutputの分割

- Stateに入力用のデータと出力用のデータが混在すると、どこでどのデータが使われているかがわかりにくい。
- `StateGraph`のinput引数とoutput引数を使って、**InputStateとOutputStateを明確に分離できる!**

In [10]:
class InputState(BaseModel):
    input_value: str


class OutputState(BaseModel):
    output_value: str


class OverallState(InputState, OutputState):
    pass


graph_builder = StateGraph(
    state_schema=OverallState,
    input=InputState,
    output=OutputState,
)


def node_A(state: InputState, config: RunnableConfig) -> OutputState:
    return OutputState(output_value=f"A + {state.input_value}")


graph_builder.add_node(node_A)
graph_builder.set_entry_point("node_A")

# グラフを実行
graph = graph_builder.compile()
print(graph.invoke(InputState(input_value="1")))

{'output_value': 'A + 1'}


## Stateを効率的に管理する3種の分割戦略2: ノード間専用のStateを定義

- **特定のノード間でのみ使用するデータをInputやOutputのStateに含めると**、全体のStateが肥大化して管理が難しくなる。
- → 「ノード間専用のState」を作成し、適切な範囲でのみ利用する！

In [11]:
class InputState(BaseModel):
    input_value: str


class OutputState(BaseModel):
    output_value: str


class OverallState(InputState, OutputState):
    pass


# 特定のノード間のみで使う中間State
class PrivateState(BaseModel):
    private_value: str


graph_builder = StateGraph(
    state_schema=OverallState,
    input=InputState,
    output=OutputState,
)


# PrivateStateを返すノード
def node_A(state: InputState, config: RunnableConfig) -> PrivateState:
    return PrivateState(private_value=f"A + {state.input_value}")


# PrivateStateを受け取り、OutputStateを返すノード
def node_B(state: PrivateState, config: RunnableConfig) -> OutputState:
    return OutputState(output_value=f"B + {state.private_value}")


graph_builder.add_node(node_A)
graph_builder.add_node(node_B)
graph_builder.set_entry_point("node_A")
graph_builder.add_edge("node_A", "node_B")


# グラフを実行
graph = graph_builder.compile()
print(graph.invoke(InputState(input_value="1")))


{'output_value': 'B + A + 1'}


## Stateを効率的に管理する3種の分割戦略3: Stateのスキーマを動的に管理

- LangGraphは、Stateの受け渡し時に、各ノードが必要なプロパティのみを受け取るようにフィルタリングできる!
- これにより、各ノードが必要なデータのみを受け取ることが可能。
  - コードの可読性と実行効率を向上できる...!
- 内部の動作 (へぇ〜知らなかった...!!:thinking:)
  - StateGraphの初期化時に、`state_schema`, `input`, `output`を登録 (内部でそれぞれ個別のスキーマ情報として保持されるらしい...!!)
  - ノード追加時 (`add_node`) に、各ノード関数のシグネチャをチェックし、必要なスキーマを登録。
  - 実行時(`invoke`)に、登録ずみのスキーマ情報を元に、必要なプロパティのみを抽出してノードに渡してくれる。